<a href="https://colab.research.google.com/github/BlazeBolt/sentiment-analyser/blob/main/sentiment-analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk scikit-learn --quiet

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd

df = pd.read_csv('data_analysis.csv')
df.head()

<ipython-input-2-aebf73a4b451>:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_analysis.csv')


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1406397735792791556,1406397735792791556,2021-06-20 05:15:24 IST,2021-06-20,05:15:24,530,16001827,spj_tweets,Society of Professional Journalists,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1406057066058313733,1406057066058313733,2021-06-19 06:41:42 IST,2021-06-19,06:41:42,530,1067308099,coastguardph,Philippine Coast Guard,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1405965100993986561,1405965100993986561,2021-06-19 00:36:16 IST,2021-06-19,00:36:16,530,167406704,yaelkohen,Yael Kohen,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1405935463374991366,1405935461042958338,2021-06-18 22:38:30 IST,2021-06-18,22:38:30,530,29206490,ieeeusa,IEEE-USA,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1405918849329991694,1405918849329991694,2021-06-18 21:32:29 IST,2021-06-18,21:32:29,530,31696962,dw_europe,DW Europe,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'@\w+|#', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['tweet'].astype(str).apply(clean_text)
df[['tweet', 'cleaned_text']].head()

,tweet,cleaned_text
0,Join @SPJ_IC at 6 p.m. EDT Tuesday for the nex...,join at pm edt tuesday for the next ictalk wit...
1,"READ: Resilience, hard work, and perseverance ...",read resilience hard work and perseverance pai...
2,How will the shift to streaming impact movie-m...,how will the shift to streaming impact moviema...
3,(2/2) to gain the aptitude to find better solu...,to gain the aptitude to find better solutions ...
4,Alexander Lukashenko is still in power in Bela...,alexander lukashenko is still in power in bela...


In [ ]:
def assign_sentiment(tweet):
    if any(word in tweet for word in ['love', 'great', 'good', 'amazing', 'awesome', 'best']):
        return 'positive'
    elif any(word in tweet for word in ['hate', 'bad', 'worst', 'terrible', 'problem']):
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['cleaned_text'].apply(assign_sentiment)


df = df[df['sentiment'] != 'neutral']
df['sentiment'].value_counts()

,count
sentiment,
positive,1636
negative,383


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9653465346534653
              precision    recall  f1-score   support

    negative       1.00      0.81      0.90        75
    positive       0.96      1.00      0.98       329

    accuracy                           0.97       404
   macro avg       0.98      0.91      0.94       404
weighted avg       0.97      0.97      0.96       404

